In [15]:
import rx
import rxsci as rs

In [16]:
source = [1, 2, 3, 4, 5, 6, 7]

rx.from_(source).pipe(
    rs.ops.multiplex(rx.pipe(
        rs.data.roll(window=3, stride=3, pipeline=rx.pipe(
            rs.math.mean(reduce=True),
        )),
    )),
).subscribe(
    on_next=print
)

2.0
5.0
